In [1]:
#rom AMOL.OBJECT_DETECTION.objectdetector import ObjectDetector
from AMOL.DESCRIPTOR.hog import HOG
#from pyimagesearch.utils import Conf
import imutils
#import argparse
import pickle
import cv2
from AMOL.OBJECT_DETECTION import HELPERS

In [13]:
import argparse
#AP = argparse.ArgumentParser()
#AP.add_argument("-FN","--FOLDER_NAME",required=True)
#AP.add_argument("-i","--image",required=True)
#AP.add_argument("-W","--WIDTH",required=True)
#AP.add_argument("-H","--HEIGHT",required=True)
#ARGS = vars(AP.parse_args())
ARGS={'WIDTH':92,'HEIGHT':32,'FOLDER_NAME':'car_side','image':'TEST.jpg'}

In [14]:
type(ARGS)

dict

In [15]:
hog = HOG(orientations=9, pixelsPerCell=(4,4),
	cellsPerBlock=(2,2), normalize=True)

In [16]:
model3 = pickle.loads((open("output/cars/"+ARGS["FOLDER_NAME"], "rb").read()))

In [17]:
image = cv2.imread(ARGS['image'])
image = imutils.resize(image, width=min(260, image.shape[1]))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [21]:
# import the necessary packages
from AMOL.OBJECT_DETECTION import HELPERS
#def detect(image, winDim, winStep=4, pyramidScale=1.5, minProb=0.7):
winDim = (int(ARGS['WIDTH']),int(ARGS['HEIGHT']))
boxes = []
probs = []
for layer in HELPERS.pyramid(gray):
    scale = image.shape[0] / float(layer.shape[0])
    for (x, y, window) in HELPERS.sliding_window(layer, 4,winDim):
        (winH, winW) = window.shape[:2]
        if winH == winDim[1] and winW == winDim[0]:
            features = hog.describe(window).reshape(-1,1)
            features = features.reshape(1,features.shape[0])
            #print(len(features),type(features))
            prob = model3.predict_proba(features)[0][1]
            if prob > 0.9:
                (startX, startY) = (int(scale * x), int(scale * y))
                endX = int(startX + (scale * winW))
                endY = int(startY + (scale * winH))
                boxes.append((startX, startY, endX, endY))
                probs.append(prob)

C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [28]:
INDEX = probs.index(max(probs))
cv2.rectangle(image, (boxes[INDEX][0], boxes[INDEX][1]), (boxes[INDEX][2], boxes[INDEX][3]), (0, 0, 255), 2)

array([[ 98, 111, 151, ...,  47,  48,  56],
       [ 79,  83, 130, ...,  50,  50,  54],
       [ 84,  88, 120, ...,  53,  51,  55],
       ...,
       [ 64,  65,  66, ...,  79,  81,  82],
       [ 65,  66,  67, ...,  79,  81,  82],
       [ 67,  67,  67, ...,  79,  81,  82]], dtype=uint8)

In [29]:
#cv2.imshow("Image", gray)
#cv2.waitKey(0)

-1

In [23]:
#for (startX, startY, endX, endY) in boxes[0:5]:
#    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
cv2.imshow("Image", image)
cv2.waitKey(0)

-1